# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

In [5]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [6]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [7]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [14]:
sum(data.race=='b'), sum(data.race=='w'), data.shape

(2435, 2435, (4870, 65))

In [16]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

data.sex

In [46]:
data.call.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: call, dtype: float32

### Question 1. What test is appropriate for this problem? Does CLT apply?
<ul>
    <li>I think we can test the difference of mean of black's and white's call, so that we can know if there's a race discrimination.
    <li>Yes, because the sample is generated randomly and independently.
</ul>

### Question 2. What are the null and alternate hypotheses?
<ul>
    <li> Null hypothesis: There's no difference between mean of black's and white's call.
    <li> Althernative hypothesis: There's difference between mean of black's and white's call.
</ul>

### Question 3. Compute margin of error, confidence interval, and p-value.
<ul>
    <li> Margin of error is: 0.0123523387151.
    <li> Condifence interval is: [0.01930185, 0.0439425]
    <li> P-value is：0.0
</ul>

In [52]:
def bootstrap_replicate_1d(data,func):
    bs_sample = np.random.choice(data,len(data))
    return func(bs_sample)

In [53]:
def draw_bs_reps(data,func,size=1):
    bs = np.empty(size)
    for i in range(size):
        bs[i] = bootstrap_replicate_1d(data,func)
    return bs

In [23]:
def diff_of_mean(x, y):
    return (np.mean(x) - np.mean(y))

In [24]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [25]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates

In [ ]:
black = data.race == 'b'
white = data.race == 'w'
call_black = data[black].call.reset_index().call
call_white = data[white].call.reset_index().call
white.head()
print(call_black.head())
print(call_white.head())

In [72]:
diff_mean = diff_of_mean(call_white,call_black)
print(diff_mean)

0.03203285485506058


In [80]:
bs_replicates = draw_bs_reps((call_white - call_black),np.mean,10000)
margin_of_error = 1.96 * np.std(call_white - call_black)/np.sqrt(len(call_white))
confidence_interval = np.percentile(bs_replicates,[2.5,97.5])
print('Margin of error is: ', margin_of_error)
print('95% Confidence interval:',confidence_interval)

Margin of error is:  0.0123523387151
95% Confidence interval: [ 0.01930185  0.0439425 ]


In [84]:
perm_replicates = draw_perm_reps(call_white,call_black,diff_of_mean,size=10000)
p = np.sum(perm_replicates >= diff_mean)/len(perm_replicates)
print('P value is:',p)

P value is: 0.0


### Question 4. Write a story describing the statistical significance in the context or the original problem.
<ul>
    <li> In this case, interview call for white-sounding name is more than black-sounding name according to the sample, but the result is by chance, and there's no significant difference statistically, because the p value is significantly small.  
</ul>

### Question 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
<ul>
    <li> Race/name is not most important factor in callback success, because there's no significant difference statistically. 
    <li> Additionally, it's slightly related to education and sex. Female is more possible to get callback than male, and higher education doesn't help.
</ul>

In [86]:
def pearson_r(x, y):
    """Compute Pearson correlation coefficient between two arrays."""
    corr_mat = np.corrcoef(x,y)
    return corr_mat[0,1]

In [ ]:
data.education

In [106]:
r_obs_edu = pearson_r(data.education, data.call)
print(r_obs_edu)
perm_replicates_edu = np.empty(10000)
# Draw replicates
for i in range(10000):
    # Permute illiteracy measurments: illiteracy_permuted
    temperature_permuted = np.random.permutation(data.education)
    # Compute Pearson correlation
    perm_replicates_edu[i] = pearson_r(temperature_permuted, data.call)
# Compute p-value: p
p_edu = np.sum(perm_replicates_edu <= r_obs_edu)/len(perm_replicates_edu)
print('p-val =', p_edu)

-0.00574779558277
p-val = 0.3335


In [91]:
r_obs_exp = pearson_r(data.yearsexp, data.call)
print(r_obs_exp)
perm_replicates_exp = np.empty(10000)
# Draw replicates
for i in range(10000):
    # Permute illiteracy measurments: illiteracy_permuted
    temperature_permuted = np.random.permutation(data.yearsexp)
    # Compute Pearson correlation
    perm_replicates_exp[i] = pearson_r(temperature_permuted, data.call)
# Compute p-value: p
p_exp = np.sum(perm_replicates_exp >= r_obs_exp)/len(perm_replicates_exp)
print('p-val =', p_exp)

0.0614364902871
p-val = 0.0001


In [ ]:
female = data.sex == 'f'
male = data.sex == 'm'
call_female = data[female].call.reset_index().call
call_male = data[male].call.reset_index().call
male.head()
print(call_female.head())
print(call_male.head())

In [100]:
diff_mean_sex = diff_of_mean(call_male,call_female)
print(diff_mean_sex)

-0.00864456593990326


In [107]:
perm_replicates_sex = draw_perm_reps(call_male,call_female,diff_of_mean,size=10000)
p = np.sum(perm_replicates_sex <= diff_mean_sex)/len(perm_replicates_sex)
print('P value is:',p)

P value is: 0.1972
